In [88]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.layers import  Input, dot, concatenate
from keras.models import Model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM

In [90]:
movies = pd.read_csv('./movielens_dataset/u.item' , header = None , sep = "|" , encoding='latin-1')
movies.columns = ['movie_id' , 'movie_title' , 'release_date' , 'video_release_date' ,
             'IMDb_URL' , 'unknown' , 'Action' , 'Adventure' , 'Animation' ,
             'Childrens' , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
             'Film_Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci_Fi' ,
             'Thriller' , 'War' , 'Western']

users = pd.read_csv('./movielens_dataset/u.user', header= None , sep = '|')
users.columns = ['user_id' , 'age' , 'gender' , 'occupation' , 'zip_code']

ratings = pd.read_csv('./movielens_dataset/u.data', header= None , sep = '\t')
ratings.columns = ['user_id' , 'movie_id' , 'rating' , 'timestamp']

In [91]:
data = ratings.merge(users , on='user_id')
data = data.merge(movies , on='movie_id')

In [92]:
data = data.sample(frac = 1)
data_train_x = np.array(data[['user_id' , 'movie_id']].values)
data_train_y = np.array(data['rating'].values)
x_train, x_test, y_train, y_test = train_test_split(data_train_x, data_train_y, test_size = 0.2, random_state = 98)
n_factors = 50
n_users = len(np.unique(data['user_id']))
n_movies = len(np.unique(data['movie_id']))

In [93]:
user_input = Input(shape=(1,))
user_embeddings = Embedding(input_dim = n_users+1, output_dim=n_factors, input_length=1)(user_input)
user_vector = Flatten()(user_embeddings)

movie_input = Input(shape = (1,))
movie_embeddings = Embedding(input_dim = n_movies+1 , output_dim = n_factors , input_length = 1)(movie_input)
movie_vector = Flatten()(movie_embeddings)

merged_vectors = concatenate([user_vector, movie_vector])
dense_layer_1 = Dense(100 , activation = 'relu')(merged_vectors)
dense_layer_3 = Dropout(.5)(dense_layer_1)
dense_layer_2 = Dense(1)(dense_layer_3)
model = Model([user_input, movie_input], dense_layer_2)

In [94]:
model.compile(loss='mean_squared_error', optimizer='adam' ,metrics = ['accuracy'] )
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 50)        47200       input_5[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 50)        84150       input_6[0][0]                    
____________________________________________________________________________________________

In [95]:
history = model.fit(
    x = [x_train[:,0] , x_train[:,1]] , 
    y =y_train , batch_size = 128 , 
    epochs = 20 , 
    validation_data = ([x_test[:,0] , x_test[:,1]] , y_test)
)

Epoch 1/20
625/625 [==============================] - 2s 2ms/step - loss: 2.1399 - accuracy: 0.0572 - val_loss: 0.9566 - val_accuracy: 0.0618
Epoch 2/20
625/625 [==============================] - 1s 2ms/step - loss: 1.1152 - accuracy: 0.0607 - val_loss: 0.9223 - val_accuracy: 0.0619
Epoch 3/20
625/625 [==============================] - 2s 2ms/step - loss: 1.0739 - accuracy: 0.0608 - val_loss: 0.9318 - val_accuracy: 0.0619
Epoch 4/20
625/625 [==============================] - 2s 3ms/step - loss: 1.0567 - accuracy: 0.0609 - val_loss: 0.9136 - val_accuracy: 0.0619
Epoch 5/20
625/625 [==============================] - 2s 3ms/step - loss: 1.0333 - accuracy: 0.0609 - val_loss: 0.9084 - val_accuracy: 0.0619
Epoch 6/20
625/625 [==============================] - 1s 2ms/step - loss: 1.0125 - accuracy: 0.0609 - val_loss: 0.9021 - val_accuracy: 0.0619
Epoch 7/20
625/625 [==============================] - 2s 3ms/step - loss: 0.9944 - accuracy: 0.0608 - val_loss: 0.8984 - val_accuracy: 0.0619
Epoch 

In [38]:
loss , val_loss , accuracy , val_accuracy = history.history['loss'],history.history['val_loss'],history.history['accuracy'],history.history['val_accuracy']

In [26]:
plt.figure(figsize = (12,10))
plt.plot( loss, 'r--')
plt.plot(val_loss, 'b-')
plt.plot( accuracy, 'g--')
plt.plot(val_accuracy,'-')
plt.legend(['Training Loss', 'Validation Loss' , 'Training Accuracy' , 'Validation Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

NameError: name 'loss' is not defined

<Figure size 864x720 with 0 Axes>

In [13]:
score = model.evaluate([x_test[:,0], x_test[:,1]], y_test)
print(np.sqrt(score))

625/625 [==============================] - 1s 1ms/step - loss: 0.8665 - accuracy: 0.0646
[0.93088483 0.2541653 ]


In [87]:
movie_ids = data_train_x[:,1]
predictions = list(enumerate(model.predict([[np.asarray([395]*len(data_train_x)), data_train_x[:,1]]]).flatten().tolist()))
predictions = sorted(predictions , key = lambda x: x[1] , reverse = True)
data_train_x[:,1][[i[0] for i in predictions]]


array([1599, 1536, 1467, ..., 1502, 1494, 1486])

In [272]:
keras_similar = data_train_x[:,1][[i[0] for i in predictions]][:20]

In [273]:
keras_similar

array([ 286, 1473,  649,  244,  211,  673,  366,  118,  460,  815,  263,
        425,  116, 1048,   95,  657,  588,  752,  662,  570])

In [274]:
[movies[movies['movie_id'] == e]['movie_title'].to_numpy()[0] for e in keras_similar]

['English Patient, The (1996)',
 'Little Princess, The (1939)',
 'Once Upon a Time in America (1984)',
 "Smilla's Sense of Snow (1997)",
 'M*A*S*H (1970)',
 'Cape Fear (1962)',
 'Dangerous Minds (1995)',
 'Twister (1996)',
 'Crossing Guard, The (1995)',
 'One Fine Day (1996)',
 'Steel (1997)',
 'Bob Roberts (1992)',
 'Cold Comfort Farm (1995)',
 "She's the One (1996)",
 'Aladdin (1992)',
 'Manchurian Candidate, The (1962)',
 'Beauty and the Beast (1991)',
 'Replacement Killers, The (1998)',
 'Somewhere in Time (1980)',
 'Wyatt Earp (1994)']